
# Práctica 3: Metaheurísticas basadas en poblaciones - Algoritmos Genéticos

<center><h3>
    Uxía Carro Tacón
</h3></center>



## Instrucciones



Esto es **Jupyter Notebook**, un documento que integra código Python en un archivo Markdown.
Esto nos permite ir ejecutando celdas de código poco a poco, así como generar automáticamente un informe bien formateado de la práctica.

Puedes añadir una celda con el botón **"Insert"** de la barra de herramentas, y cambiar su tipo con **"Cell > Cell Type"**

Para ejecutar una celda de código, la seleccionaremos y pulsaremos el botón **"▶ Run"** de la barra de herramentas.
Para pasar el documento a HTML, seleccionaremos **"File > Download as > HTML (.html)"**

Sigue este guión hasta el final. Ejecuta el código proporcionado paso a paso comprendiendo lo que estás haciendo y reflexionando sobre los resultados. Habrá preguntas intercaladas a lo largo del guión, responde a todas ellas en la sección reservada para ese fin: **"Respuestas a los cuestionarios"**. Por favor, no modifiques ninguna linea de código excepto cuando se te pida explícitamente.

No olvides insertar tu **nombre y apellidos** en la celda superior.

IMPORTANTE: Escribe el código de tu o tus soluciones/respuestas en las celdas que se indican para ello. Además, a lo largo de la práctica se plantearán varias preguntas que debéis responder en la parte inferior del documento, incluyendo las celdas que veáis necesarias (si hacéis referencia a partes concretas de vuestro código, etc) para reponder a ellas.

## Entrega de la práctica

La fecha límite de entrega será la indicada en el Campus Virtual. La entrega consistirá de un único archivo comprimido con nombre `APELIDOS_NOME_Geneticos.zip` que contenga los seguientes ficheros:

 * `APELIDOS_NOME_Geneticos.html`: Archivo HTML fruto de la exportación del presente Notebook, con las preguntas respondidas al final del documento.
 * `APELIDOS_NOME_Geneticos.ipynb`: Archivo fuente Jupyter Notebook.
 * Archivo de datos de los problema utilizados en la resolución.


## Preliminares adicionales sobre Python


Para esta práctica, te sugerimos algunas funciones de paquetes que pueden resultarte útiles en la realización de esta práctica.

Has visto que cuando realizas laboratorios y pruebas para ajustar parámetros resulta necesario tener una estimación del tiempo invertido en la ejecución. Eso ayuda a valorar el balance del tiempo computacional frente a la inclusión de valores de parámetros en el laboratorio. En ese sentido, puede resultar conveniente añadir barras de progreso que además te permitan visualizar el progreso de resolución de las iteraciones. Para ese propósito podéis hacer uso del paquete `tqdm` (https://tqdm.github.io/). 

Puedes ver ejemplos en detalle descritos en https://towardsdatascience.com/progress-bars-for-python-with-tqdm-4dba0d4cb4c

Veámos un ejemplo ilustrativo aquí.

In [9]:
# solo debes importarlo una vez en el notebook.
# fíjate que estamos importando del paquete tqdm.notebook para que incorpore los decoradores compatibles en Jupyter 
# (en Python podrías importar simplemente desde el paquete
import tqdm.notebook

ModuleNotFoundError: No module named 'tqdm'

`tqdm` permite añadir una barra de progreso que informe sobre el tiempo y el paso de iteraciones. En el ejemplo, la variable `nit` son el número de iteraciones a realizar y para conseguir que se visualize el progreso se pasa a `tqdm` un iterador de rango.

In [10]:
from random import randint

heads = 0
tails = 0
nit=1000000
for i in tqdm(range(nit), desc='Coin Flip Progress'):
    toss = randint(0, 1)
    if toss == 0:
        heads += 1
    else:
        tails += 1

NameError: name 'tqdm' is not defined

El paquete nos ofrece la posibilidad combinar `tdqm(range(NUM_IT))` en una única función, llamada `trange(NUM_IT)`. Veamos su funcionamiento en el siguiente ejemplo, que muestra también cómo podemos hacer cuando necesitamos llevar cuenta del progreso en bucles anidados.

In [11]:
num_games = 3

for game in trange(num_games, desc='Overall Progress'):
    heads = 0
    tails = 0
    nit=1000000
    for j in trange((nit), desc=f'Game {game+1} Progress'):
        toss = randint(0, 1)
        if toss == 0:
            heads += 1
        else:
            tails += 1
            
    print(f'Heads: {heads}, Tails: {tails}')


NameError: name 'trange' is not defined

Aprovechamos para recordarte que puedes utilizar semillas en la generación de secuencias de números aleatorios para hacer determinista y más verificable tu implementación. 

In [1]:
import random
import time

In [13]:
# este vector será aleatorio si no habíamos establecido una semilla previamente.
# Podríamos tener incluso una semilla basada en el tiempo actual para forzar que se aleatorice 
# si ejecutamos esta celda múltiples veces
random.seed(time.time())

vector_aleatorio = [ random.randint(1, 10) for i in range(1,10) ]
print ("vector aleatorio ", vector_aleatorio)

# aquí establecemos una semilla totalmente fija
# que no depende del tiempo y por tanto está bajo nuestro control
semilla = 123456
random.seed (semilla)

# este viene determinado por la semilla
vector_aleatorio = [ random.randint(1, 10) for i in range(1,10) ]
print ("vector aleatorio ", vector_aleatorio)

# y ahora somos capaces de generar la misma serie "aleatoria"
random.seed (semilla)
vector_aleatorio = [ random.randint(1, 10) for i in range(1,10) ]
print ("vector aleatorio igual que el anterior", vector_aleatorio)


vector aleatorio  [9, 10, 1, 8, 2, 7, 8, 5, 2]
vector aleatorio  [5, 1, 3, 1, 2, 1, 5, 1, 2]
vector aleatorio igual que el anterior [5, 1, 3, 1, 2, 1, 5, 1, 2]


Esto puede ser útil para verificar que siempre obtienes los mismos resultados ante el mismo conjunto de entradas.

## El Problema del Viajante de Comercio (VC) con Algoritmos Genéticos

El objetivo de esta práctica es modelar e implementar un agente inteligente que sea capaz de resolver el problema del VC mediante la metaheurística (MH) de poblaciones conocida como Algoritmo Genético (GA, del inglés *Genetic  Algorithm*). Para ello, realizarás una implementación del algoritmo básico visto en la clase expositiva y valorarás si la introducción de modificaciones en el diseño del algoritmo te permite mejorar la calidad de las soluciones alcanzadas.


### Definición del problema de Viajante de Comercio (VC)


La definición del problema sigue la descripción ya vista en las prácticas anterior, en la que se disponía de una represetación del problema mediante un grafo ponderado. Así que, en primer lugar importa el módulo Python que acompaña esta práctica, que ya trae las funciones de apoyo como la clase `Localizaciones` que implementaba la carga de datos y que utilizaba una matriz de adyacencia.

In [1]:
from helpers_mod_ga import *

Si todo va bien, la primera distancia entre la ciudad 0 y 1 debe ser unos 55 km para el problema de las 8 ciudades gallegas.

In [2]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))

55.88273580792048



## P3.1: Implementación básica de Algoritmo Genético


Implementa un algoritmo genético básico para resolver el problema del VC, siguiendo la descripción algorítmica de la MH vista en la clase expositiva.

Ten en cuenta las siguientes consideraciones de diseño para completar esta implementación básica:

- Representación  de  las  soluciones:  representación  de  orden  (permutaciones) comenzando y finalizando en la ciudad 0. 

- Población inicial: inicialización completamente aleatoria de la población inicial, que sean permutaciones válidas.

- Operador de cruce: Order Crossover (OX). La función/método que se encargue de hacer el cruce ordenado debe aceptar como parámetros la probabilidad de cruce (por defecto, establecida a pc=0.95), los dos puntos de corte así como los cromosomas sobre los que actuar.

- Operador de mutación: se utilizará el operador de intercambio, aplicado con una probabilidad de mutación. Vuestra implementación debe manejar dos variantes de esta función:
    - Mutación individual: solo se aplica la mutación, si procede, sobre un único gen en el cromosoma. La probabilidad de mutación por defecto se establece pm=0.25. Esta función debería recibir el índice del gen sobre el que actuar además de la probabilidad de mutación y el cromosoma original.
    - Mutación cromosómica: se aplica la mutación contemplando todo el cromosoma, de forma que individualmente puede mutar cada gen de manera independiente según una probabilidad de mutación, establecida por defecto a pm=0.01.

- El reemplazo de la generación debe seguir un modelo de reemplazo generacional con elitismo. En particular, tu implementación debe disponer de dos variantes de la función de reemplazo:
    - mitades: haz que la mitad de la población resultante del reemplazo provenga de la generación actual mientras que la otra mitad provenga de los descendientes, tomando en ambos casos los mejores individuos. 
    - elitismo "elite=2": de esta forma, hay que mantener los 2 mejores individuos de la generación actual/ancestros; mientras que el resto provienen de entre los mejores hijos. La función de reemplazo debería parametrizar "elite".
    

Ambas variantes deben recibir como parámetro la lista de cromosomas de las poblaciones ascentros e hijos generados y retornar la población resultante.

- Condición de parada simple basado en un número de iteraciones máximo pasado como parámetro.

- La implementación debe ser completamente parametrizable, de forma que todos los componentes susceptibles de ser ajustados mediante parámetros deberían puedan ser establecidos en cada ejecución/problema.

Lanza varias ejecuciones para verificar que puede resolver el problema con los siguientes grafos de
ejemplo:

- grafo 8 ciudades gallegas (`data/grafo8cidades.txt`), con una población de 2 individuos y 150 iteraciones.
- grafo 120 ciudades estadounidenses (`data/US120.txt`), con una población de 32 individuos y 500 iteraciones.

Si no te convergiera, prueba a extender el número de iteraciones o variar parámetros como la probabilidad de mutación.



In [3]:
# Escribe aquí tu código para la función que implenta tu algoritmo genético
# Crea tantas celdas como consideres oportuno para escribir tu código
# Documenta siempre tu código con comentarios como este

import random
import time

#funcion para generar una solucion inicial
def genera_solucion_inicial(longitud):
    random.seed(time.time())
    #se obtiene un array con las ciudades intermedias mezcladas
    vector_aleatorio = random.sample(list(range(1, longitud)), longitud-1)
    return vector_aleatorio

#funcion para calcular el coste de una solución: la distancia total entre las ciudades de la solución
def coste(g, solucionI):
    solucion = [0]+solucionI+[0]
    suma_coste = 0
    #se van sumando las distancias entre las ciudades de la solucion
    for i in range(0, len(solucion)-1):
        suma_coste += g.distancia(solucion[i], solucion[i+1])
    return suma_coste


#operador de cruce para hacer el cruce ordenado de los cromosomas 1 y 2 con 2 puntos de corte. El cruce sucede con una probabilidad pc
def operador_cruce(puntoCorte1, puntoCorte2, cromosoma1, cromosoma2, pc=0.95):
    #se encarga de hacer el cruce ordenado 
    #entre dos cromosomas
    #se obtienen los cromosomas intermedios
    random.seed(time.time())
    prob = random.random()
    if (prob > pc):
        return cromosoma1, cromosoma2
    
    #hace el cruce ordenado de manera que no se repitan los genes en los cromosomas
    #se obtienen los genes de los cromosomas
    genes1 = cromosoma1[puntoCorte1:puntoCorte2]
    genes2 = cromosoma2[puntoCorte1:puntoCorte2]
    #se hace el cruce ordenado
    #se obtienen los genes que no estan en el cromosoma2
    genes_faltantes = [x for x in cromosoma2[puntoCorte2:] if x not in genes1]
    for x in cromosoma2[:puntoCorte2]:
        if x not in genes1:
            genes_faltantes.append(x)
    hijo1 = [0] * len(cromosoma1)
    hijo1[puntoCorte1:puntoCorte2] = genes1
    #rellena H1 con el resto de genes en el orden en el que aparecen en el cromosoma 2 a partir del segundo punto de corte
    for i in range(puntoCorte2, len(hijo1)):
        if (hijo1[i] == 0): hijo1[i] = genes_faltantes.pop()
    for i in range(puntoCorte1):
        if (hijo1[i] == 0): hijo1[i] = genes_faltantes.pop()
    
    genes_faltantes = [x for x in cromosoma1[puntoCorte2:] if x not in genes2]
    for x in cromosoma1[:puntoCorte2]:
        if x not in genes2:
            genes_faltantes.append(x)
    hijo2= [0]*len(cromosoma2)
    hijo2[puntoCorte1:puntoCorte2] = genes2
    for i in range(puntoCorte2, len(hijo2)):
        if(hijo2[i]==0): hijo2[i] = genes_faltantes.pop()
    for i in range(puntoCorte1):
        if(hijo2[i]==0): hijo2[i] = genes_faltantes.pop()
    
    #se devuelve los cromosomas
    return hijo1, hijo2


#operador de mutacion mediante intercambio. Ocurre con una probabilidad de mutacion pm. Se aplica sobre un único gen del cromosoma
#recibe el índice del gen, el cromosoma individual y la pm
def mutacion_individual(cromosoma, gen, pm=0.25):
    random.seed(time.time())
    prob = random.random()
    if (prob > pm):
        return cromosoma
    #si se muta, producimos un descendiente ligeramente modificado
    #se escoge una posicion aleatoria del cromosoma
    random.seed(time.time())
    posicion = random.randint(0, len(cromosoma)-1)
    while (gen == posicion):
        random.seed(time.time())
        posicion = random.randint(0, len(cromosoma)-1)
    cromosoma = list(cromosoma)
    #se intercambian los genes
    aux = cromosoma[posicion]
    cromosoma[posicion] = cromosoma[gen]
    cromosoma[gen] = aux

    return cromosoma

#operador de mutacion cromosomica mediante intercambio. Probabilidad pm
# se aplica la mutacion contemplando todo el cromosoma, de forma que individualmente se puede mutar cada gen según la probabilidad pm
def mutacion_cromosomica(cromosoma, pm=0.01):
    #para cada gen del cromosoma se decide si se muta o no
    for i in range(0, len(cromosoma)):
        random.seed(time.time())
        if (random.random() > pm):
            #se pasa al siguiente gen
            continue
        random.seed(time.time())
        posicion = random.randint(0, len(cromosoma)-1)
        cromosoma[i], cromosoma[posicion] = cromosoma[posicion], cromosoma[i]
    return cromosoma

#reemplazo de la generacion con elitismo
#mitades: la mitad de la poblacion resultante proviene de hijos y la otra mitad de ancestros, de ambos se cogen los mejores individuos
def reemplazo_generacion_mitades(g, ancestros, hijos, tam_poblacion):
    ancestros_o = sorted(ancestros, key=lambda x: coste(g, x))
    hijos_o = sorted(hijos, key=lambda x: coste(g, x))
    #se seleccionan los mejores de los ancestros y de los hijos
    mejores = ancestros_o[:int(tam_poblacion/2)] + hijos_o[:int(tam_poblacion/2)]
    return mejores

#elite=2. Mantenemos los 2 mejores individuos de la generacion anterior. El resto de la poblacion se reemplaza por los mejores descendientes
def reemplazo_generacion_elite(g, ancestros, hijos, tam_poblacion, elite=2):
    ancestros_o = sorted(ancestros, key=lambda x: coste(g, x))
    hijos_o = sorted(hijos, key=lambda x: coste(g, x))
    #se seleccionan los 2 mejores de los ancestros 
    mejores = ancestros_o[:elite]
    for i in range(len(hijos_o)):
        if (len(mejores) == tam_poblacion):
            break
        mejores.append(hijos_o[i])
    return mejores

#funcion para seleccionar los puntos de corte para el operador de cruce
def puntos_corte(g):
    random.seed(time.time())
    punto1 = random.randint(0, g.nciudades-1)
    random.seed(time.time())
    punto2 = random.randint(0, g.nciudades-1)
    while (punto1 == punto2):
        punto2 = random.randint(0, g.nciudades-1)
    if (punto1 > punto2):
        punto1, punto2 = punto2, punto1
    return punto1, punto2

In [4]:
def algoritmo_genetico(tam_poblacion, n_generaciones, mitades=0, individual=0, pmi=0.25, pmc=0.01, pc=0.95, elite=2, filename="./data/grafo8cidades.txt"):
    #se genera una poblacion inicial de tamaño tam_poblacion de manera aleatoria
    poblacion = []
    g = Localizaciones(filename=filename)
    for i in range(0, tam_poblacion):
        poblacion.append(genera_solucion_inicial(g.nciudades))
    iteracion=0
    while (iteracion < n_generaciones): #condicion de parada
        hijos = []
        punto1, punto2 = puntos_corte(g)
        #desordenamos la población para coger los padres de dos en dos sin tener en cuenta el coste
        #porque al hacer el reemplazo se ordenan según coste
        random.seed(time.time())
        random.shuffle(poblacion)
        for i in range(0, tam_poblacion-1, 2):
            #seleccion de padres a cruzar
            
            hijo1, hijo2 = operador_cruce(punto1, punto2, poblacion[i], poblacion[i+1], pc)
            if (individual == 0):
                hijos.append(mutacion_cromosomica(hijo1, pmc))
                hijos.append(mutacion_cromosomica(hijo2, pmc))
            else:
                random.seed(time.time())
                gen = random.randint(0, g.nciudades-2)
                hijos.append(mutacion_individual(hijo1, gen, pm=pmi))
                random.seed(time.time())
                gen = random.randint(0, g.nciudades-2)
                hijos.append(mutacion_individual(hijo2, gen,pm=pmi))
        #print('HIJOS: ', hijos)
        if (mitades == 0):
            poblacion = reemplazo_generacion_elite(g, poblacion, hijos, tam_poblacion, elite)
        else:
            poblacion = reemplazo_generacion_mitades(g, poblacion, hijos, tam_poblacion)
        iteracion += 1
    
    #se devuelve el mejor individuo de la poblacion
    poblacion = sorted(poblacion, key=lambda x: coste(g, x))
    return poblacion[0], coste(g, poblacion[0])

In [5]:
operador_cruce(3, 7, [1, 7, 2, 5, 3, 6, 4], [7, 4, 1, 2, 6, 5, 3])

([2, 1, 7, 5, 3, 6, 4], [4, 7, 1, 2, 6, 5, 3])

In [10]:
g = Localizaciones(filename="./data/grafo8cidades.txt")
sol = genera_solucion_inicial(g.nciudades)
print(g.nciudades, sol)

8 [1, 2, 3, 5, 6, 7, 4]


In [11]:
algoritmo_genetico(2, 150, mitades=0, individual=1)

([3, 2, 6, 4, 1, 5, 7], 721.0678274993777)

In [85]:
algoritmo_genetico(2, 150, mitades=1)

([7, 6, 5, 4, 3, 2, 1], 381.6699617675482)

In [47]:
#con elitismo
sol120 = algoritmo_genetico(32, 500, mitades=0, filename="./data/US120.txt")
print(sol120[1])

87285.984474142


In [49]:
#con mitades
sol120_o = algoritmo_genetico(32, 500, mitades=1, filename="./data/US120.txt")
print(sol120_o)

([69, 7, 66, 28, 93, 75, 27, 60, 86, 111, 15, 118, 12, 115, 106, 67, 4, 3, 53, 34, 117, 41, 14, 55, 64, 11, 18, 37, 2, 82, 10, 79, 32, 59, 47, 20, 26, 113, 84, 119, 80, 101, 114, 107, 105, 71, 35, 94, 56, 30, 29, 33, 108, 97, 44, 6, 45, 76, 102, 85, 48, 57, 87, 103, 96, 92, 52, 65, 63, 95, 72, 46, 104, 74, 43, 77, 68, 100, 51, 91, 50, 58, 16, 1, 25, 88, 73, 49, 70, 9, 23, 17, 24, 62, 21, 83, 99, 98, 61, 31, 116, 39, 112, 54, 5, 19, 36, 42, 89, 8, 110, 109, 13, 81, 22, 90, 78, 38, 40], 67207.12051242735)


In [50]:
print(sol120_o[1])

67207.12051242735


In [39]:
operador_cruce(1, 3, [1, 2, 3, 4, 5, 6], [1, 3, 5, 6, 4, 2])

([5, 2, 3, 6, 4, 1], [2, 3, 5, 4, 6, 1])

❓ **Pregunta 1**. Explica brevemente los detalles relevantes de tu código para entender tu implementación (p.ej., estructura de tu código, funciones, implementación de operadores, etc.). ¿Cuál es el mecanismo de selección escogido para tu implementación? Explica también cómo has verificado tu implementación.



## P3.2: Laboratorio

Como toda metaheurística, resulta crucial conocer cómo afecta en términos prácticos los parámetros
del algoritmo y los operadores desarrollados. Estudia el comportamiento del algoritmo implementado para resolver VC atendiendo a las siguientes cuestiones.


❓ **Pregunta 2**. Realiza el estudio de la calidad de la solución variando el tamaño de la población: 1, 2, 4, 8, 16, 32, 64... ¿Qué valor recomendarías para el problema de las 120 ciudades? ¿Por qué? Ten en cuenta que tienes dos variantes del mecanismo de reemplazo.

❓ **Pregunta 3**. Ahora realiza un estudio similar variando la probabilidad de mutación en el rango 0, ..., 0.95 en pasos de 0.05~0.10. ¿Qué valor recomendarías para este parámetro? ¿Por qué? Ten en cuenta que tienes dos variantes del mecanismo de mutación.

❓ **Pregunta 4**. Finalmente realiza un estudio variando el número de iteraciones máximas en el rango de 50 a 1000, considerando un paso variable o adaptativo a conveniencia. ¿Qué valor recomendarías para este parámetro? ¿Por qué? Realiza una discusión reflexionando sobre los resultados conjuntos de estos tres parámetros. 

Se espera q a medida q dejemos evolcionar, debería seguir mejorando. Si vemos q tras x iteraciones no se mejora o empeora, cortar y no continuar iterando. Discutir como estos 3 parámetros, decidir con cual nos quedamos de cada cosa.

Apoya todas tus respuestas en datos-gráficos resultantes de tus estudios, reportados de forma que sea fácil de comparar/contrastar.

Importante: además de la calidad de la soluciones obtenidas, se recomienda medir tiempos para tomar medidas operativas sobre el número de repeticiones que permitan realizar promedios (se recomienda no más de 10 si se ralentiza mucho) y prescindir de manera razonada de valores en las series de ejecución que no sean computacionalmente rentables/viables con tu implementación/ordenador (p.ej., tamaños de población elevados pueden tomar mucho tiempo para resolver).



In [51]:
# puedes añadir aquí la implementación o cualquier código que necesites para responder las preguntas
# utiliza tantas celdas como necesites


Calidad de la solución variando el tamaño de la población:

In [13]:
soluciones_mitades8=[]
soluciones_elitismo8=[]
for i in range(2, 100, 2):
    soluciones_elitismo8.append(algoritmo_genetico(i, 150, mitades=0, individual=1, filename='./data/grafo8cidades.txt')[1])
    soluciones_mitades8.append(algoritmo_genetico(i, 150, mitades=1, individual=1, filename='./data/grafo8cidades.txt')[1])

In [6]:
soluciones_mitades120=[]
soluciones_elitismo120=[]
tiempos_mitades=[]
tiempos_elitismo=[]
for i in range(2, 100, 2):
    ti = time.time()
    soluciones_elitismo120.append(algoritmo_genetico(i, 150, mitades=0, individual=1, filename='./data/US120.txt')[1])
    tf = time.time()
    tiempos_elitismo.append(tf-ti)
    ti = time.time()
    soluciones_mitades120.append(algoritmo_genetico(i, 150, mitades=1, individual=1, filename='./data/US120.txt')[1])
    tf = time.time()
    tiempos_mitades.append(tf-ti)
    
soluciones_mitades120c=[]
soluciones_elitismo120c=[]
tiempos_mitadesc=[]
tiempos_elitismoc=[]
for i in range(2, 100, 2):
    ti = time.time()
    soluciones_elitismo120c.append(algoritmo_genetico(i, 150, mitades=0, individual=0, filename='./data/US120.txt')[1])
    tf = time.time()
    tiempos_elitismoc.append(tf-ti)
    ti = time.time()
    soluciones_mitades120c.append(algoritmo_genetico(i, 150, mitades=1, individual=0, filename='./data/US120.txt')[1])
    tf = time.time()
    tiempos_mitadesc.append(tf-ti)


In [9]:
print('Minimo mitades individual: ', min(soluciones_mitades120))
print('Minimo elitismo individual: ', min(soluciones_elitismo120))

Minimo mitades individual:  75192.02986772107
Minimo elitismo individual:  110479.85640946431


Calidad de la solución variando la probabilidad de mutación, tanto para la variación individual como para la cromosómica:

In [22]:
si=[]
sc=[]
tin=[]
tc=[]
for i in range(1, 20, 1):
    v = i*0.05
    ti = time.time()
    si.append(algoritmo_genetico(64, 200, mitades=1, individual=1, pmi=v, filename='./data/US120.txt')[1])
    tf = time.time()
    tin.append(tf-ti)
    ti = time.time()
    sc.append(algoritmo_genetico(64, 200, mitades=1, individual=0, pmc=v, filename='./data/US120.txt')[1])
    tf = time.time()
    tc.append(tf-ti)

In [7]:
s=[]
t=[]
for i in range(50, 1000, 50):
    ti=0
    tf=0
    v = i*0.05
    ti = time.time()
    s.append(algoritmo_genetico(64, i, mitades=1, individual=1, pmi=0.18, filename='./data/US120.txt')[1])
    tf = time.time()
    t.append(tf-ti)

In [8]:
print(s)
print(t)

[113071.92855422349, 90148.38757295441, 80349.97313836917, 76770.12020303392, 68306.60276774346, 70340.5737098801, 63188.522130729594, 64170.69689579273, 62587.13613729509, 58228.41394221328, 56731.177481846345, 56128.964178729686, 55927.14056422645, 49783.38411135381, 56458.871928276436, 54072.964948834284, 48390.39665252764, 47752.59503146575, 50820.705491460656]
[0.6186819076538086, 1.2716822624206543, 1.784712553024292, 2.224792003631592, 2.6210057735443115, 3.3548405170440674, 3.804619312286377, 4.174257755279541, 4.622564315795898, 5.274459362030029, 5.7522547245025635, 6.3693482875823975, 7.047818899154663, 7.2383177280426025, 7.7624523639678955, 8.59170913696289, 8.954241752624512, 9.501838445663452, 10.516944169998169]



#### Respuestas y evaluación

Recordatorio: No olvides escribir tu nombre y apellidos en la segunda celda de este documento.
La respuestas a las preguntas deben venir acompañadas de las implementaciones necesarias para su respuesta.

*P3.1: Implementación básica* (5 puntos)

La implementación básica se evaluará mediante un cuestionario automático de evaluación. Este lo realizarás en la primera sesión tras la entrega, y se centrará en la resolución por tu parte de diversas cuestiones prácticas relacionadas con la implementación realizada, pudiendo ser necesaria la ejecución, adaptación y modificación de la misma. 

Pregunta 1.

Independientemente del cuestionario automático de evaluación, considera siempre que las preguntas planteadas en el notebook deben ser respondidas también. Esas preguntas generales están diseñadas para formarte, y te servián para razonar y reflexionar sobre el tema, así como también para fomentar una discusión constructiva con los docentes en caso de dudas.


*P3.2: Laboratorio* (5 puntos)

Pregunta 2.

Pregunta 3.

Pregunta 4.


El informe a elaborar no debe exceder la longitud máxima de 1200 palabras.

Aclaraciones: La evaluación de esta parte se llevará a cabo en términos de la completitud y correctitud del laboratorio implementado, así como de la calidad del propio informe, que debe ser conciso y preciso, pudiendo acompañarse de gráficas y tablas que faciliten y fundamenten la explicación e argumentación. Es muy importante explicar de manera clara, precisa y fundamentada. Se reservará hasta un punto que se asignará en términos de la calidad de la mejor solución obtenida entre el conjunto de las prácticas entregadas (es por ello que no debes olvidar marcar en tu informe muy claramente cuál ha sido tu mejor solución y con qué configuración/versión).



## Informe

Vamos estudiar el comportamiento del algoritmo para resolver el problema del viajante de comercio para 120 ciudades estadounidenses, cuyos datos podemos obtener de './data/US120.txt'.

**Estudio de la calidad de la solución variando el tamaño de la población.**

En primer lugar, queremos obtener el mejor valor de tamaño de población para la resolución del problema con estas 120 ciudades. 

Para la realización de este estudio se han utilizado las dos variaciones para la mutación, la individual y la cromosómica,  así como las dos variaciones de reemplazo de la generación implementadas, reemplazo por mitades y mediante elitismo, con élite=2. Además, se han utilizado 150 iteraciones.

Con los parámetros especificados obtenemos los siguientes resultados. En las dos primeras gráficas podemos ver los resultados en coste y tiempo de computación para la variante de mutación individual, y en las dos siguientes los de la mutación cromosómica.

<img src="./fotos/graf3.png">

Con estos datos, podemos ver que sobre todo para el caso de mitades, el coste va disminuyendo de manera gradual a medida que aumentamos el tamaño de población. En cuanto al tiempo de computación, se va elevando a medida que aumenta el tamaño de población, como era de esperar. Esto lo hace de una manera bastante uniforme, con algunos pequeños picos debidos a las probabilidades de mutación y cruce, que no ocurren en todos los casos. 

También se puede apreciar fácilmente que el comportamiento de los dos tipos de mutación es parecido, tanto en la evolución del coste, como del tiempo de computación necesario, a pesar de que para el caso de la mutación cromosómica es notablemente más elevado, puesto que se realizan muchas más operaciones. 

Una vez observados estos datos podemos ver claramente que se puede llegar a una buena solución sin un tamaño de población excesivamente elevado. Sin embargo, el comportamiento de la variantes de reemplazo es diferente, excepto para el tamaño de población 4, en el que ambas variaciones se comportarán de la misma manera, puesto que seleccionarán los 2 mejores padres y los 2 mejores hijos obtenidos. Además, con el tamaño de población 2, para el caso de elitismo con el parámetro élite = 2 no va a tener sentido puesto que nunca se van a reemplazar los dos padres obtenidos con la inicialización aleatoria, por eso se obtiene un coste mucho más elevado.

Puesto que cada una de las variaciones se comportará de manera diferente, no tiene sentido elegir un mismo valor para el tamaño de población para ambos. 

En cuanto a la variante de reemplazo con elitismo, con un valor de elite=2, parece que con el aumento del tamaño de la población, se van obteniendo soluciones ligeramente peores, por lo que para no desperdiciar tiempo de computación estaría bien quedarnos con un tamaño de población cercano a 16. Esto sucede porque al aumentar el tamaño de la población pero manteniendo el mismo valor de élite, la proporción de soluciones que mantenemos de una generación a la siguiente se va haciendo mucho más pequeña, seleccionando únicamente las 2 mejores y completando el resto de la población de la siguiente generación con los nuevos hijos generados, que pueden ser peores en cuanto a coste que muchos de los ancestros descartados, que ya no se emplearán para crear hijos nuevos y posiblemente mejores.

Por otra parte, para el reemplazo por mitades, sí que se puede observar que el coste disminuye cada vez más, por lo que sería bueno elegir un tamaño más elevado, como 64, con el que obtendríamos una buena solución sin incrementar excesivamente el tiempo necesario.


**Calidad de la solución variando la probabilidad de mutación:**

En este apartado se probará de manera experimental el comportamiento del algoritmo genético variando las probabilidades de mutación. La mutación ocurre tras el cruce de los padres seleccionados con una probabilidad dada. Contamos con dos variantes de esta mutación: la individual, en la cual se intercambia un único gen del cromosoma con otro aleatorio; y la cromosómica, para la que se exploran todos los genes.

Las variantes se probarán por separado con valores entre 0.05 y 0.95, incrementandose en 0.05. Además utilizaremos únicamente el reemplazo por mitades, para el que utilizaremos un tamaño de población de 64. Para el criterio de parada utilizaremos 200 iteraciones. Con estos parámetros obtenemos los siguientes resultados:

<img src='./fotos/graf4.png'>

En las anteriores gráficas podemos ver en azul las variaciones del coste de la solución obtenida para cada caso probado, y en rojo el tiempo de computación empleado. 

En estas vemos que para ambas mutaciones el coste obtenido se va haciendo cada vez mayor a medida que aumentamos la probabilidad de mutación, a la vez que también lo hace el tiempo requerido, ya que el hecho de que la mutación ocurra requiere de más operaciones, lo que se va haciendo cada vez más probable. En cuanto al coste de la solución final, encontramos diferencias entre el comportamiento cada variante. 

Para la mutación individual vemos una trayectoria ascendente pero con notables picos, debidos a que el parámetro modificado es una probabilidad por lo que no determina completamente el comportamiento del algoritmo. Aquí se intercambia con la probabilidad dada un único gen del cromosoma de cada hijo, por lo que en el caso de que se dé, el cromosoma seguirá siendo parecido a sus antecesores. Sin embargo, a medida que la probabilidad aumenta, la mutación se va haciendo menos controlada, haciendo la búsqueda demasiado explroatoria, con lo que se dificulta la convergencia hacia mejores soluciones.

En el caso de la mutación cromosómica, esto último que comentábamos se hace mucho más notorio, puesto que en lugar de intercambiarse un único gen de los hijos, pueden intercambiarse cada uno de ellos con la probabilidad establecida. Esto hace que aumentando la probabilidad de que ocurra, los hijos pueden llegar a ser completamente diferentes a los padres sin seguir ningún criterio más allá de la aleatoriedad. Por esto mismo, las soluciones obtenidas empeoran mucho más rápido.

Para obtener unas buenas soluciones tenemos que contemplar un equilibrio entre la exploracion y la explotación, manteniendo una mutación controlada, por lo que las probabilidades escogidas deben ser bajas, sobre todo la de la cromosómica. Así, nos quedaremos con una probabilidad de 0.18 para la individual y 0.05 para la cromosómica, aunque también sería bueno contemplar valores aún más bajos.


**Comportamiento del algoritmo genético variando el número de iteraciones máximas:**

Por último en este informe estudiaremos la influencia del número de iteraciones máximas, para lo que usaremos entonces los valores obtenidos hasta ahora, con tamaño de población 64, reemplazo por mitades y mutación individual con probabilidad 0.18. Con estos parámetros obtenemos las siguientes soluciones:

<img src='./fotos/graf5.png'>

En estos resultados se puede ver que el coste disminuye rápidamente al aumentar el número de iteraciones, puesto que el algoritmo tiene más oportunidades de encontrar mejores soluciones y explotarlas. Al mismo tiempo, el coste aumenta de manera prácticamente lineal. 

Puesto que se trata de una metaheurística, la finalidad es obtener una solución suficientemente buena en un tiempo razonable. Por esta razón, para el número de iteraciones nos quedaremos con 750, que junto con el resto de valores decididos en los anteriores apartados conseguimos un coste de **49783.3841km** en **7.2383 segundos**.